# Object detection evaluation

Test a Detector on A Customized Dataset (here, I use the Faster RCNN)


In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
import matplotlib.pyplot as plt
import cv2
import os
from detectron2.data.datasets import register_coco_instances

## Load the model

In [ ]:
# model path
model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_Flux_SSL/checkpoint/train_weights/RN50_25K_200e/Exp1/SSL_20per_5/model_best_78.4433.pth"
model_config_path = "/scratch/tjian/PythonProject/deep_plastic_Flux_SSL/checkpoint/train_weights/RN50_25K_200e/Exp1/SSL_20per_5/config.yaml"

cfg = get_cfg()
cfg.merge_from_file(model_config_path)
cfg.MODEL.WEIGHTS = model_checkpoint_path 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

## Register my custom test dataset

dataset should be in COCO format

In [ ]:
# Register a test dataset
register_coco_instances("Flux_test", {}, "/scratch/tjian/Data/Flux/TUD_Vietnam/1209/annotations/1209.json", "/scratch/tjian/Data/Flux/TUD_Vietnam/1209/images_with_litter/")
Test_Dataset_name="Flux_test"
Class_name=["litter"]

## Prediction

### (1) Predict one image

In [ ]:
im = cv2.imread("/scratch/tjian/Data/Jakarta_coco/val/Grogol_Location_1_2018-04-30 16-00-00-05_cam6.jpg")
outputs = predictor(im)
# print(outputs) # It will print num_instances, box location, scores, prediction labels

v = Visualizer(im,
               MetadataCatalog.get(Test_Dataset_name).set(thing_classes=Class_name),
               instance_mode=ColorMode.SEGMENTATION,
               scale=1.0
               )

out = v.draw_instance_predictions(outputs["instances"].to("cpu"))  # do not write "gpu"
img=out.get_image()
result=cv2.imwrite(r"/scratch/tjian/Data/Jakarta_coco/Predictions_SSL/predict.jpg", img)


### (2) Predict images in a folder and output No.predictions

In [ ]:
import os

# define the folder path 
folder_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/images_no_litter/"

# define output folder of predicted images
out_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Pred/SSL_100per/No_SA/images_no_litter/"

# The total number of detections
predictions = 0

file_names = os.listdir(folder_path)
for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            out_file_name= os.path.join(out_path, filename)
            im = cv2.imread(image_path)
            # start predicting
            outputs = predictor(im)
            print(outputs)
            print(len(outputs["instances"].get("pred_boxes")))
            predictions = predictions + len(outputs["instances"].get("pred_boxes"))
            v = Visualizer(im,
                           MetadataCatalog.get(Test_Dataset_name).set(thing_classes=Class_name),
                           instance_mode=ColorMode.SEGMENTATION,
                           scale=1.0
                           )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))  # do not write "gpu"
            img = out.get_image()
            result = cv2.imwrite(out_file_name, img)

print("Done")
print("The total number of detections: ", predictions)

### (3) Test on a test dataset with labels

Run a model to predict images with ground-truth labels and output the accuracy, e.g., AP50

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Test
save_evaluate_file_dir = r"/scratch/tjian/PythonProject/deep_plastic_Flux_SSL/checkpoint/train_weights/RN50_500K/Exp1/RN50_500K_200e/SSL_20per_7/test_results"

# create the "save_evaluate_file_dir" if they do not exist
if not os.path.isdir(save_evaluate_file_dir):
    os.mkdir(save_evaluate_file_dir)


evaluator = COCOEvaluator(Test_Dataset_name, output_dir=save_evaluate_file_dir)
val_loader = build_detection_test_loader(cfg, Test_Dataset_name)
print(inference_on_dataset(predictor.model, val_loader, evaluator))

### (4) Output accuracy according to prediction and ground-truth files

Compare predictions file (json file) and ground-truth files (json file), and output the accuracy, e.g., AP50

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Load ground truth annotations
coco_gt = COCO('/scratch/tjian/Data/GJO_SSL/test_1/test_PR/labels_coco/test.json')

# Load detection results
coco_dt = coco_gt.loadRes('/scratch/tjian/Data/GJO_SSL/test_1/test_PR/SSL_pred_results_NMS_0.5/coco_instances_results.json')

# Create COCOEval object
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')

# Run evaluation
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Get precision and recall values at different IoU thresholds
precision = coco_eval.eval['precision']
recall = coco_eval.eval['recall']

### (5) Output confusion matrix on test dataset with annotations, i.e., TP, FP, FN

See the "main_Confusion_matrix_OD.ipynb"


### (6) Calculate the FP on a dataset without litter annotations

In [ ]:
import os

# define the folder path 
folder_path = r"/scratch/tjian/Data/Borgharen_test/labeled_data/0110/images_no_litter/"

file_names = os.listdir(folder_path)
FP = 0
for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            im = cv2.imread(image_path)
            # start predicting
            outputs = predictor(im)
            # print(outputs)
            # print(len(outputs["instances"].get("pred_boxes")))
            FP = FP + len(outputs["instances"].get("pred_boxes"))

print("FP: ", FP)
